In [1]:

import os,sys
sys.path.append(os.path.abspath("."))                    # repo_root
sys.path.append(os.path.abspath("aside/experiments")) # to access aside.experiments
from aside.experiments.model import *
from aside.experiments.model_api import *
import torch
from transformers import AutoTokenizer,AutoModelForCausalLM
import json
import numpy as np
import pickle

from datasets import load_dataset
from tqdm import tqdm
from collections import defaultdict,Counter
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import sys
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)
from utils.utils import *
from copy import deepcopy
from utils.steering import *
torch.set_grad_enabled(False)
from struq import *
from config import *
from eval import *
from utils.torch_hooks import *
from utils.probe import *
# home_dir = "/home/wjyeo/IPI_defense" # change for home
home_dir = "/export/home2/weijie210/StruQ"
# home_dir = '..'

cache_dir = f'{home_dir}/cache'
os.makedirs(cache_dir,exist_ok=True)
seed_all()

[2025-09-01 09:53:53,156] [INFO] [real_accelerator.py:254:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/export/home2/weijie210/miniconda3/envs/cot/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/export/home2/weijie210/miniconda3/envs/cot/compiler_compat/ld: warning: libstdc++.so.6, needed by /cm/shared/apps/cuda12.1/toolkit/12.1.1/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/export/home2/weijie210/miniconda3/envs/cot/compiler_compat/ld: warning: libm.so.6, needed by /cm/shared/apps/cuda12.1/toolkit/12.1.1/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/export/home2/weijie210/miniconda3/envs/cot/compiler_compat/ld: /cm/shared/apps/cuda12.1/toolkit/12.1.1/lib64/libcufile.so: undefined reference to `std::runtime_error::~runtime_error()@GLIBCXX_3.4'
/export/home2/weijie210/miniconda3/envs/cot/compiler_compat/ld: /cm/shared/apps/cuda12.1/toolkit/12.1.1/lib64/libcufile.so: undefined reference to `__gxx_personality_v0@CXXABI_1.3'
/export/home2/weijie210/miniconda3/envs/cot/compiler_compat/ld: /cm/

In [ ]:
import glob
def load_hf_model(model_name,is_work = False): # only for work
    if is_work:
        user,model_name = model_name.split('/')
        base = f"../../huggingface_models/models--{user}--{model_name}/snapshots"
        commit_hash = os.path.basename(glob.glob(f"{base}/*")[0])  # first snapshot folder
        model_path = os.path.join(base, commit_hash)
        return model_path
    else:
        data_dir = '/dataset/common/huggingface/model'
        
        return os.path.join(data_dir, model_name)
    
device = 'cuda:0'
torch_dtype = torch.bfloat16
# model_name = "Qwen3-8B-ASIDE"
model_name = "Qwen3-8B_SpclSpclSpcl_None"
model_path = load_hf_model(model_name)  # set to_work to true if running on work

if 'SpclSpclSpcl' in model_path: # load as per normal
    tokenizer = AutoTokenizer.from_pretrained(model_path) # change to model path at work
    tokenizer.padding_side = "left"
    tokenizer.model_max_length = 1024

    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        torch_dtype=torch_dtype,
        low_cpu_mem_usage=True,
        trust_remote_code=True,
    ).to(device).eval()
    model.tokenizer = tokenizer
    

else: # use aside code
    base_model_path = 'Qwen/Qwen3-8B' # change to model path at work
    handler = CustomModelHandler(
        model_path, base_model_path, base_model_path, model_path, None,
        0, embedding_type='forward_rot' if '8b-aside' in model_path.lower() else 'ise',
        load_from_checkpoint=True,model_dtype=torch_dtype,
    )
    handler.model.to(device).eval()
    model = handler.model
    model.tokenizer = handler.tokenizer
    tokenizer = handler.tokenizer
    handler.max_token_len = 1024

num_layers = len(model.model.layers)
num_heads = model.config.num_attention_heads

def get_model_name(model_path):
    if '8b-aside' in model_path.lower():
        return 'ASIDE'
    elif 'NaiveCompletion' in model_path:
        return 'StruQ'
    elif '8b-ise' in model_path.lower():
        return 'ISE'
    elif 'none' in model_path.lower():
        return 'Vanilla'

model.name = get_model_name(model_path)

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [4]:
from aside.experiments.model_api import format_prompt as aside_fp # not to confuse with utils.format_prompt

def eval_mcq(model,tokenizer,dataset,bz,format_fn,avg= True):
    acc = []
    for i in tqdm(range(0,len(dataset),bz),total = len(dataset)//bz):
        batch = dataset[i:i+bz]
        answer = [d['answer'] for d in batch]
        prompts = format_fn(batch)
        with torch.no_grad():
            outputs = model(**prompts).logits[:,-1]
        pred = tokenizer.batch_decode(outputs.argmax(dim=-1),skip_special_tokens=True)
        acc.extend([p.lower() == a.lower() for p,a in zip(pred,answer)])
    return np.mean(acc) if avg else acc

import evaluate
rouge = evaluate.load("rouge")

def eval_oe(model,tokenizer,dataset,bz,format_fn,avg= True,max_new_tokens=64):
    em = []
    rouge_scores = [] # take rogueL
    for i in tqdm(range(0,len(dataset),bz),total = len(dataset)//bz):
        batch = dataset[i:i+bz]
        answer = [d['answer'] for d in batch]
        prompts = format_fn(batch)
        with torch.no_grad():
            outputs = model.generate(**prompts,max_new_tokens=max_new_tokens)[:,prompts['input_ids'].shape[1]:]
        pred = tokenizer.batch_decode(outputs,skip_special_tokens=True)
        if not isinstance(answer[0],list):
            em.extend([a.lower() in p.lower() for a,p in zip(answer,pred)])
            rouge_scores.extend([rouge.compute(predictions = [p], references = [a])['rougeL'] for p,a in zip(pred,answer)])
        else:
            for ans,p in zip(answer,pred):
                em.append(any([a.lower() in p.lower() for a in ans]))
                rouge_scores.append(rouge.compute(predictions = [p], references = ans)['rougeL'])
            

    return np.mean(em) if avg else em, np.mean(rouge_scores) if avg else rouge_scores

## Settle the format fn
if model.name.lower() == 'aside':

    with open('aside/experiments/data/prompt_templates.json', 'r') as f:
        templates = json.load(f)
    template = templates[0]

    def def_format_fn(samples,attack_fn= lambda x: x,suffix = ''): # def the forward pass for aside model, add a suffix
        samples = deepcopy(samples)
        inst_list,data_list = [],[]
        for sample in samples:
            sample = attack_fn(sample)
            instruction_text = aside_fp(sample['instruction'], template, "system")
            data_text = aside_fp(sample['input'], template, "user")
            inst_list.append(instruction_text)
            data_list.append(data_text)
        return {k:v.to(model.device) for k,v in handler.call_model_api_batch(inst_list, data_list,max_new_tokens=1024,do_sample = False,return_inputs_only=True,suffix=suffix).items()}

    mcq_ff = partial(def_format_fn,suffix='The answer is (') # force the first token to be done.
    nr_mcq_ff = partial(no_rotation,format_fn=mcq_ff)

    oe_ff = partial(def_format_fn,suffix='Response:') # open ended, no suffix
    nr_oe_ff = partial(no_rotation,format_fn=oe_ff)
else:
    def vanilla_format(samples,tokenizer,attack_fn = lambda x: x,apply_defensive_filter=True,suffix = ''): # formatting for vanilla/struq model
        prompt_format = PROMPT_FORMAT["SpclSpclSpcl"]
        formatted_inputs = form_llm_input(samples,attack_fn,prompt_format,apply_defensive_filter=apply_defensive_filter,defense = 'none') 
        formatted_inputs = [x + suffix for x in formatted_inputs]
        return tokenizer(formatted_inputs, return_tensors="pt", padding='longest', truncation=False).to(model.device)

    mcq_ff = partial(vanilla_format,tokenizer=tokenizer,suffix='The answer is (') # force the first token to be done.
    oe_ff = partial(vanilla_format,tokenizer=tokenizer) # open ended, no suffix

In [ ]:
race_ds = load_dataset("ehovy/race", "high",split = 'test').to_list()
filtered_race_ds = []
for d in race_ds:
    d['instruction'] = d.pop('question')
    d['input'] = d.pop('article')
    choices = '\n'.join([f"({chr(i+65)}) {c}" for i,c in enumerate(d['options'])])
    d['instruction'] = f'Instruction: {d["instruction"]}\n\nChoices:\n{choices}'
    input_len = mcq_ff([d])['input_ids'].shape[1] # get the length of the input
    if input_len < 512: # if longer than model's length then ignore.
        filtered_race_ds.append(d)
print (len(filtered_race_ds),len(race_ds))
race_ds = filtered_race_ds


reclor_ds = load_dataset("metaeval/reclor",split = 'validation').to_list()

filtered_reclor_ds = []
for d in reclor_ds:
    d['instruction'] = d.pop('question')
    d['input'] = d.pop('context')
    choices = '\n'.join([f"({chr(i+65)}) {c}" for i,c in enumerate(d['answers'])])
    d['instruction'] = f'Instruction: {d["instruction"]}\n\nChoices:\n{choices}'
    d['answer'] = chr(d['label'] + 65)
    input_len = mcq_ff([d])['input_ids'].shape[1] # get the length of the input
    if input_len < 512: # if longer than model's length then ignore.
        filtered_reclor_ds.append(d)
print (len(filtered_reclor_ds),len(reclor_ds))
reclor_ds = filtered_reclor_ds

newsqa_ds = load_dataset("lucadiliello/newsqa",split = 'validation').to_list()

filtered_newsqa_ds = []
for d in newsqa_ds:
    d['instruction'] = d.pop('question')
    d['input'] = d.pop('context')
    d['answer'] = d.pop('answers')
    input_len = mcq_ff([d])['input_ids'].shape[1] # get the length of the input
    if input_len < 512: # if longer than model's length then ignore.
        filtered_newsqa_ds.append(d)
print (len(filtered_newsqa_ds),len(newsqa_ds))
newsqa_ds = filtered_newsqa_ds

Token indices sequence length is longer than the specified maximum sequence length for this model (1025 > 1024). Running this sequence through the model will result in indexing errors


2476 3498
499 500
1085 4212


In [7]:
utility_results = defaultdict(dict) if model.name.lower() == 'aside' else {}

In [8]:
bz = 64
for k,v in {'race':race_ds,'reclor':reclor_ds}.items():
    aside_acc = eval_mcq(model, tokenizer, v, bz, mcq_ff,False)
    
    if model.name.lower() == 'aside':
        nr_aside_acc = eval_mcq(model, tokenizer, v, bz, nr_mcq_ff,False)

        print (f'{k.upper()} ASIDE/no rotation acc: {np.mean(aside_acc):.4f}/{np.mean(nr_aside_acc):.4f}')
        utility_results[k]['normal'] = np.mean(aside_acc)
        utility_results[k]['no_rotation'] = np.mean(nr_aside_acc)
    else:
        utility_results[k] = np.mean(aside_acc)
        print (f'{k.upper()} {model.name} acc: {np.mean(aside_acc):.4f}')

39it [01:43,  2.65s/it]                                                                             


RACE Vanilla acc: 0.6817


8it [00:23,  2.98s/it]                                                                              

RECLOR Vanilla acc: 0.5070


Open-ended on newsqa

In [9]:
bz = 64
aside_em,aside_rouge = eval_oe(model,tokenizer,newsqa_ds,bz,oe_ff,False)
if model.name.lower() == 'aside': # do no_rotation as well
    nr_side_em,nr_aside_rouge = eval_oe(model,tokenizer,newsqa_ds,bz,nr_oe_ff,False)
    print (f'ASIDE/no rotation em: {np.mean(aside_em):.2f}/{np.mean(nr_side_em):.2f}, rouge: {np.mean(aside_rouge):.2f}/{np.mean(nr_aside_rouge):.2f}')
    utility_results['newsqa']['normal'] = {'em':np.mean(aside_em),
                                        'rouge':np.mean(aside_rouge)}
    utility_results['newsqa']['no_rotation'] = {'em':np.mean(nr_side_em),
                                        'rouge':np.mean(nr_aside_rouge)}
else:
    utility_results['newsqa'] = {'em':np.mean(aside_em),
                                        'rouge':np.mean(aside_rouge)}
    print (f'{model.name} em: {np.mean(aside_em):.2f}, rouge: {np.mean(aside_rouge):.2f}')

100%|███████████████████████████████████████████████████████████████| 16/16 [03:22<00:00, 13.30s/it]Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
17it [03:35, 12.68s/it]                                                                             

Vanilla em: 0.54, rouge: 0.23


In [10]:
print (tokenizer.decode(mcq_ff(newsqa_ds[:1])['input_ids'][0]))

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

[MARK] [INST][COLN]
What will be nominated?

[MARK] [INPT][COLN]
(CNN) -- What could be more powerful than the tears of a Native American Indian?



Wax on, wax off: Does it make you want to save the rainforests?



Iron Eyes Cody was the face of the Keep American Beautiful campaign of 1971 whose tears marked the plight of the environment, but more importantly kept the problems of pollution in the minds of millions.



From teary Native Americans to witty skits or doom-ladened eco-horror scenarios, the environmental campaign video then has long been a powerful tool for environmental groups to spread their message and raise pubic attention.



The rise of YouTube and other video sharing web sites has now meant that individuals can broadcast their own eco-awareness messages and form their own social action networks.



But what mak

In [15]:
utility_results['newsqa']['em'] = np.mean(utility_results['newsqa']['em'])
utility_results['newsqa']['rouge'] = np.mean(utility_results['newsqa']['rouge'])

In [16]:
result_cache = f'results/utility/{model.name}.json'
os.makedirs(os.path.dirname(result_cache), exist_ok=True)

with open(result_cache,'w') as f:
    json.dump(utility_results,f,indent=4)
